In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

Setting environment for Gordon


No vtk


In [6]:
stack_fixed = 'MD603'

In [5]:
train_sample_scheme = 1
global_transform_scheme = 1

# stack_moving = 'atlas_on_MD589'
stack_moving = 'atlasV2'

In [4]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

structures_sided = sum([[n] if n in singular_structures 
                        else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

In [8]:
label_to_name_fixed = {i+1: name for i, name in enumerate(sorted(structures))}
name_to_label_fixed = {n:l for l, n in label_to_name_fixed.iteritems()}

In [9]:
# label_to_name_moving = {i+1: name for i, name in enumerate(sorted(structures) + sorted([s+'_surround' for s in structures]))}
# name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

In [10]:
label_to_name_moving = {i+1: name for i, name in enumerate(sorted(structures_sided) + sorted([s+'_surround' for s in structures_sided]))}
name_to_label_moving = {n:l for l, n in label_to_name_moving.iteritems()}

In [11]:
volume_fixed = {name_to_label_fixed[name]: DataManager.load_score_volume(stack=stack_fixed, label=name, downscale=32, train_sample_scheme=train_sample_scheme)
               for name in structures}

print volume_fixed.values()[0].shape
print volume_fixed.values()[0].dtype

# vol_fixed_xmin, vol_fixed_ymin, vol_fixed_zmin = (0,0,0)
# vol_fixed_ymax, vol_fixed_xmax, vol_fixed_zmax = np.array(volume_fixed.values()[0].shape) - 1

(421, 654, 399)
float16


In [12]:
# volume_moving = {name_to_label_moving[name]: DataManager.load_score_volume(stack_moving, label=name, downscale=32)
#                  for name in structures + [s+'_surround' for s in structures]}

volume_moving = {name_to_label_moving[name]: DataManager.load_score_volume(stack=stack_moving, label=name, downscale=32, train_sample_scheme=None)
               for name in structures_sided}

print volume_moving.values()[0].shape
print volume_moving.values()[0].dtype

# vol_moving_xmin, vol_moving_ymin, vol_moving_zmin = (0,0,0)
# vol_moving_ymax, vol_moving_xmax, vol_moving_zmax = np.array(volume_moving.values()[0].shape) - 1

(342, 418, 371)
float16


In [13]:
volume_moving_structure_sizes = {l: np.count_nonzero(vol > 0) for l, vol in volume_moving.iteritems()}

In [14]:
def convert_to_original_name(name):
    return name.split('_')[0]

labelIndexMap_m2f = {}
for label_m, name_m in label_to_name_moving.iteritems():
    labelIndexMap_m2f[label_m] = name_to_label_fixed[convert_to_original_name(name_m)]

label_weights_m = {}
for label_m, name_m in label_to_name_moving.iteritems():
    if 'surround' in name_m:
        label_weights_m[label_m] = 0
    else:
#         label_weights_m[label_m] = 1
        label_weights_m[label_m] = np.minimum(1e5 / volume_moving_structure_sizes[label_m], 1.)

In [16]:
aligner = Aligner4(volume_fixed, volume_moving, labelIndexMap_m2f=labelIndexMap_m2f)

In [17]:
aligner.set_centroid(centroid_m='volume_centroid', centroid_f='volume_centroid')
# aligner.set_centroid(centroid_m='structure_centroid', centroid_f='centroid_m', indices_m=[name_to_label_moving['SNR_R']])

In [18]:
gradient_filepath_map_f = {ind_f: DataManager.get_score_volume_gradient_filepath_template(stack=stack_fixed, label=label_to_name_fixed[ind_f],
                            downscale=32, train_sample_scheme=train_sample_scheme)
                            for ind_m, ind_f in labelIndexMap_m2f.iteritems()}

In [19]:
aligner.load_gradient(gradient_filepath_map_f=gradient_filepath_map_f, indices_f=None)

set([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])


load gradient 1: 1.310661 seconds
load gradient 2: 1.977060 seconds
load gradient 3: 1.760638 seconds
load gradient 4: 1.514809 seconds
load gradient 5: 1.823989 seconds
load gradient 6: 1.243615 seconds
load gradient 7: 1.930981 seconds
load gradient 8: 1.523559 seconds
load gradient 9: 1.478452 seconds
load gradient 10: 1.799599 seconds
load gradient 11: 2.921514 seconds
load gradient 12: 2.198878 seconds
load gradient 13: 1.563735 seconds
load gradient 14: 2.318128 seconds
load gradient 15: 1.653548 seconds
load gradient 16: 2.202204 seconds
load gradient 17: 2.682532 seconds
load gradient 18: 2.470826 seconds
load gradient 19: 3.034634 seconds
load gradient 20: 2.894266 seconds
load gradient 21: 2.686151 seconds
load gradient 22: 2.441745 seconds
load gradient 23: 2.726368 seconds
load gradient 24: 2.379899 seconds
load gradient 25: 1.320701 seconds
load gradient 26: 2.033009 seconds
load gradient 27: 2.351011 seconds
load gradient 28: 2.797821 seconds
overall: 59.044498 seconds


In [20]:
# largely the same optimization path regardless of the starting condition

# For rigid, 
# grad_computation_sample_number = 1e5 is desired
# grid_search_iteration_number and grid_search_sample_number seem to be unimportant as well, set to 100
# lr1=10, lr2=.1 is best

# For affine, 
# lr2 = .001 is too slow; 0.1 rises faster than 0.01
# lr1 does not matter
# plateus around iteration 100, but keep rising afterwards.
# grad_computation_sample_number does not make a difference

while True:

    try:

        if global_transform_scheme == 1:

            T, scores = aligner.optimize(type='affine', max_iter_num=1000, history_len=10, terminate_thresh=1e-4,
#                                          indices_m=[name_to_label_moving['SC'], name_to_label_moving['IC'],
#                                                    name_to_label_moving['SC_surround'], name_to_label_moving['IC_surround']],
                                         indices_m=None,
                                        grid_search_iteration_number=30,
                                         grid_search_sample_number=100,
                                         grad_computation_sample_number=1e5,
                                         lr1=10, lr2=0.1,
                                         label_weights=label_weights_m,
                                        std_tx=50, std_ty=50, std_tz=100, std_theta_xy=np.deg2rad(10))

        elif global_transform_scheme == 2:

            T, scores = aligner.optimize(type='rigid', max_iter_num=1000, history_len=10, terminate_thresh=1e-4,
                                         indices_m=None,
                                        grid_search_iteration_number=30,
                                         grid_search_sample_number=100,
                                         grad_computation_sample_number=1e5,
                                         lr1=10, lr2=0.1,
                                        label_weights=label_weights_m,
                                        std_tx=50, std_ty=50, std_tz=100, std_theta_xy=np.deg2rad(10))
        
        break
        
    except Exception as e:
        sys.stderr.write(e.message + '\n')


Regularization weights not set, default to 0.
grid search: 5.833147 seconds
-inf 0.916499
grid search: 3.624322 seconds
grid search: 3.332715 seconds
grid search: 2.638975 seconds
0.916499 0.977554
grid search: 1.822301 seconds
0.977554 1.005150
grid search: 1.422947 seconds
grid search: 1.123301 seconds
1.005150 1.005823
grid search: 1.022294 seconds
1.005823 1.008513
grid search: 0.720714 seconds
1.008513 1.010285
grid search: 0.616055 seconds
grid search: 0.619479 seconds
1.010285 1.012186
grid search: 0.617445 seconds
1.012186 1.013272
grid search: 0.622824 seconds
grid search: 0.618043 seconds
grid search: 0.607561 seconds
grid search: 0.598534 seconds
grid search: 0.611199 seconds
grid search: 0.611800 seconds
grid search: 0.610994 seconds
grid search: 0.603438 seconds
grid search: 0.608207 seconds
grid search: 0.606431 seconds
grid search: 0.618916 seconds
grid search: 0.617732 seconds
grid search: 0.616751 seconds
grid search: 0.611715 seconds
grid search: 0.611705 seconds
grid

params_best_upToNow [ 7.1690801   4.20344239 -7.17065361 -0.02057486]


step: 2.55 seconds
score: 1.013272
iteration 1
step: 2.54 seconds
score: 0.988771
iteration 2
step: 2.53 seconds
score: 1.069093
iteration 3
step: 2.52 seconds
score: 1.060606
iteration 4
step: 2.53 seconds
score: 1.114060
iteration 5
step: 2.52 seconds
score: 1.074313
iteration 6
step: 2.52 seconds
score: 1.100388
iteration 7
step: 2.52 seconds
score: 1.183932
iteration 8
step: 2.53 seconds
score: 1.133573
iteration 9
step: 2.52 seconds
score: 1.146255
iteration 10
step: 2.52 seconds
score: 1.258399
iteration 11
step: 2.52 seconds
score: 1.270383
iteration 12
step: 2.53 seconds
score: 1.273965
iteration 13
step: 2.52 seconds
score: 1.271382
iteration 14
step: 2.53 seconds
score: 1.280779
iteration 15
step: 2.53 seconds
score: 1.236230
iteration 16
step: 2.53 seconds
score: 1.282274
iteration 17
step: 2.53 seconds
score: 1.229222
iteration 18
step: 2.54 seconds
score: 1.290962
iteration 19
step: 2.53 seconds
score: 1.282236
iteration 20
step: 2.54 seconds
score: 1.305472
iteration 21
s

In [15]:
print T.reshape((3,4))
plt.plot(scores);
print max(scores), scores[-1]

NameError: name 'T' is not defined

In [22]:
params_fp = DataManager.get_global_alignment_parameters_filepath(stack_moving=stack_moving,
                                                                stack_fixed=stack_fixed,
                                                                train_sample_scheme=train_sample_scheme,
                                                                global_transform_scheme=global_transform_scheme)


DataManager.save_alignment_parameters(params_fp, T, 
                                      aligner.centroid_m, aligner.centroid_f,
                                      aligner.xdim_m, aligner.ydim_m, aligner.zdim_m, 
                                      aligner.xdim_f, aligner.ydim_f, aligner.zdim_f)

In [23]:
score_plot_fp = DataManager.get_global_alignment_score_plot_filepath(stack_moving=stack_moving,
                                                                    stack_fixed=stack_fixed,
                                                                    train_sample_scheme=train_sample_scheme,
                                                                    global_transform_scheme=global_transform_scheme)

fig = plt.figure();
plt.plot(scores);
plt.savefig(score_plot_fp, bbox_inches='tight')
plt.close(fig)

# Read

In [24]:
# stack_fixed = 'MD591'
# stack_moving = 'atlas_on_MD589'
stack_moving = 'atlasV2'

In [25]:
global_params, centroid_m, centroid_f, xdim_m, ydim_m, zdim_m, xdim_f, ydim_f, zdim_f = \
DataManager.load_global_alignment_parameters(stack_moving=stack_moving,
                                                    stack_fixed=stack_fixed,
                                                    train_sample_scheme=train_sample_scheme,
                                                    global_transform_scheme=global_transform_scheme)
#                                             trial_idx=1)

In [ ]:
xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = \
DataManager.load_score_volume_bbox(stack=stack_fixed, label='7N', downscale=32)

print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

In [15]:
# volumes_annotation = {'MD594':bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_atlasAlignParams_atlas_v2/MD594_to_MD589/MD594_down32_annotationVolume_alignedTo_MD589_down32_annotationVolume.bp'),
#                       'MD589': bp.unpack_ndarray_file(VOLUME_ROOTDIR + '/MD589/MD589_down32_annotationVolume.bp')}

In [20]:
volumes_annotation = {'MD594': bp.unpack_ndarray_file(DataManager.get_transformed_volume_filepath(stack_m='MD594', type_m='annotation',
                                                stack_f='MD589', type_f='annotation',
                                                downscale=32, global_transform_scheme=global_transform_scheme)),
                      
                      'MD589': bp.unpack_ndarray_file(DataManager.get_annotation_volume_filepath(stack='MD589', downscale=32))}

In [21]:
from registration_utilities import transform_volume

In [22]:
annotation_volumes_volume_m_aligned_to_f = {}

for stack, volume_annotation in volumes_annotation.iteritems():
    
    annotation_volumes_volume_m_aligned_to_f[stack] = transform_volume(vol=volume_annotation, 
                                                                       global_params=global_params, 
                                                                       centroid_m=centroid_m, 
                                                                       centroid_f=centroid_f,
                                                                      xdim_f=xdim_f,
                                                                      ydim_f=ydim_f,
                                                                      zdim_f=zdim_f)
    
    output_fn = DataManager.get_transformed_volume_filepath(stack_m=stack, type_m='annotation',
                                                stack_f=stack_fixed, type_f='score',
                                                downscale=32, train_sample_scheme_f=train_sample_scheme)
    
    create_if_not_exists(os.path.dirname(output_fn))

    bp.pack_ndarray_file(annotation_volumes_volume_m_aligned_to_f[stack], output_fn)

In [ ]:
# atlas_on_MD589

In [94]:
from registration_utilities import find_contour_points

In [95]:
downsample_factor = 32
xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
voxel_z_size = section_thickness / xy_pixel_distance_downsampled

In [96]:
viz_dir = create_if_not_exists(DataManager.get_global_alignment_viz_dir(stack_moving=stack_moving,
                                                        stack_fixed=stack_fixed,
                                                        train_sample_scheme=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme))

In [97]:
first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]

In [28]:
stack_colors = {'MD589': (255,0,0), 'MD594': (0,255,0)}

In [29]:
bar = show_progress_bar(first_sec, last_sec)

for sec in range(first_sec, last_sec+1):
    
    if metadata_cache['sections_to_filenames'][stack_fixed][sec] in ['Placeholder', 'Rescan', 'Nonexisting']:
        continue
    
    bar.value = sec

    img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
    img = imread(img_fn)
    
    viz = img.copy()
    
    z = voxel_z_size * (sec - 1) - zmin_vol_f
    
    # Find fixed volume annotation contours
#     contours_f_on_volume = find_contour_points(volume_fixed[..., int(z)])
#     contours_f_on_cropped = {i: [cnt + (xmin_vol_f, ymin_vol_f) for cnt in cnts] for i, cnts in contours_f_on_volume.iteritems()}

    # Find moving volume annotation contours
    
    for stack, volume_m_aligned_to_f in annotation_volumes_volume_m_aligned_to_f.iteritems():
        contours_m_alignedTo_f_on_volume = find_contour_points(volume_m_aligned_to_f[..., int(z)])
        contours_m_alignedTo_f_on_cropped = {i: [cnt + (xmin_vol_f, ymin_vol_f) for cnt in cnts] 
                                             for i, cnts in contours_m_alignedTo_f_on_volume.iteritems()}

    #     # Draw fixed volume annotation contours
    #     for ind_f, cnts_f in contours_f_on_cropped.iteritems():
    #         for cnt_f in cnts_f:
    #             cv2.polylines(viz, [cnt_f.astype(np.int)], True, (0,255,0), 2)

        # Draw moving volume annotation contours
        for ind_m, cnts_m in contours_m_alignedTo_f_on_cropped.iteritems():
            for cnt_m in cnts_m:
                cv2.polylines(viz, [cnt_m.astype(np.int)], True, stack_colors[stack], 2)

    viz_fn = os.path.join(viz_dir, '%(stack_moving)s_to_%(stack_fixed)s_%(sec)04d.jpg' % \
          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'sec': sec})
    imsave(viz_fn, viz)

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


# Transform probabilistic atlas
This part should go to `transform_brains`

In [ ]:
# Transform probabilistic atlas - atlasV2 
# Identical to the part at the bottom of this notebook.

for name_s in structures_sided:
    
    print name_s
    
    vol = bp.unpack_ndarray_file(DataManager.get_score_volume_filepath(stack=stack_moving, label=name_s, downscale=32))
    
    vol_m_aligned_to_f = transform_volume(vol=vol, global_params=global_params, 
                        centroid_m=centroid_m, 
                        centroid_f=centroid_f,
                        xdim_f=xdim_f,
                        ydim_f=ydim_f,
                        zdim_f=zdim_f)
    
    vol_m_aligned_to_f = fill_sparse_score_volume(vol_m_aligned_to_f)
    
    output_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                                stack_f=stack_fixed, type_f='score',
                                                label=name_s,
                                                downscale=32, 
                                                train_sample_scheme_f=train_sample_scheme,
                                                global_transform_scheme=global_transform_scheme)
    
    create_if_not_exists(os.path.dirname(output_fn))
    bp.pack_ndarray_file(vol_m_aligned_to_f, output_fn)

# Visualize overlay

In [7]:
# Read transformed volumes

vols_m_aligned_to_f = {name_s: DataManager.load_transformed_volume(stack_m=stack_moving, type_m='score',
                                         stack_f=stack_fixed, type_f='score',
                                         label=name_s, downscale=32,
                                         train_sample_scheme_f=train_sample_scheme,
                                         global_transform_scheme=global_transform_scheme)
                        for name_s in structures_sided}

IOError: [Errno 2] No such file or directory: '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes2/atlasV2_to_MD603/atlasV2_down32_scoreVolume_to_MD603_down32_scoreVolume_5N_L_trainSampleScheme_1_globalTxScheme_1.bp'

In [116]:
level_colors = {level: (int(level*255),0,0) for level in [0.1, 0.25, 0.5, 0.75, .99]}

In [ ]:
viz_dir = create_if_not_exists(DataManager.get_global_alignment_viz_dir(stack_moving=stack_moving,
                                                        stack_fixed=stack_fixed,
                                                        train_sample_scheme=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme))

In [117]:
from registration_utilities import find_contour_points

# estimate mapping between z and section
downsample_factor = 32
xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
voxel_z_size = section_thickness / xy_pixel_distance_downsampled

first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]
bar = show_progress_bar(first_sec, last_sec)

for sec in range(first_sec, last_sec+1):
    
    if metadata_cache['sections_to_filenames'][stack_fixed][sec] in ['Placeholder', 'Rescan', 'Nonexisting']:
        continue
    
    bar.value = sec

    img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
    img = imread(img_fn)
    
    viz = img.copy()
    
    z = voxel_z_size * (sec - 1) - zmin_vol_f
    
    # Find moving volume annotation contours
    for name_s, vol in vols_m_aligned_to_f.iteritems():
        for level in [0, 0.25, 0.5, 0.75, .99]:
            cnts = find_contours(vol[..., z], level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                

    viz_fn = os.path.join(viz_dir, '%(stack_moving)s_to_%(stack_fixed)s_%(sec)04d.jpg' % \
          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'sec': sec})
    imsave(viz_fn, viz)

/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
# TRANSFORM VOLUMES - TODO: put in a separate file

In [19]:
from registration_utilities import transform_volume

In [25]:
# Transform moving volume, sided

structures_sided = sum([[n] if n in singular_structures else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

for name_s in structures_sided:
    
    print name_s
    
    vol_m = DataManager.load_score_volume(stack=stack_moving, label=name_s, downscale=32)
    
    volume_m_alignedTo_f = \
    transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                      xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)
    
    volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                            stack_f=stack_fixed, type_f='score',
                                            label=name_s,
                                            downscale=32, 
                                            train_sample_scheme_f=1)
    
    create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
    
    bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

5N_L
5N_R
6N_L
6N_R
7N_L
7N_R
7n_L
7n_R
Amb_L
Amb_R
LC_L
LC_R
LRt_L
LRt_R
Pn_L
Pn_R
Tz_L
Tz_R
VLL_L
VLL_R
RMC_L
RMC_R
SNC_L
SNC_R
SNR_L
SNR_R
3N_L
3N_R
4N_L
4N_R
Sp5I_L
Sp5I_R
Sp5O_L
Sp5O_R
Sp5C_L
Sp5C_R
PBG_L
PBG_R
10N_L
10N_R
VCA_L
VCA_R
VCP_L
VCP_R
DC_L
DC_R
AP
12N
RtTg
SC
IC


In [20]:
# Transform moving volume, sided

structures_sided = sum([[n] if n in singular_structures else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

for name_s in structures_sided:
    
    print name_s+'_surround'
    
    vol_m = DataManager.load_score_volume(stack=stack_moving, label=name_s+'_surround', downscale=32)
    
    volume_m_alignedTo_f = \
    transform_volume(vol=vol_m, global_params=global_params, centroid_m=centroid_m, centroid_f=centroid_f,
                      xdim_f=xdim_f, ydim_f=ydim_f, zdim_f=zdim_f)
    
    volume_m_alignedTo_f_fn = DataManager.get_transformed_volume_filepath(stack_m=stack_moving, type_m='score',
                                            stack_f=stack_fixed, type_f='score',
                                            label=name_s+'_surround',
                                            downscale=32, 
                                            train_sample_scheme_f=1)
    
    create_if_not_exists(os.path.dirname(volume_m_alignedTo_f_fn))
    
    bp.pack_ndarray_file(volume_m_alignedTo_f, volume_m_alignedTo_f_fn)

5N_L
5N_R
6N_L
6N_R
7N_L
7N_R
7n_L
7n_R
Amb_L
Amb_R
LC_L
LC_R
LRt_L
LRt_R
Pn_L
Pn_R
Tz_L
Tz_R
VLL_L
VLL_R
RMC_L
RMC_R
SNC_L
SNC_R
SNR_L
SNR_R
3N_L
3N_R
4N_L
4N_R
Sp5I_L
Sp5I_R
Sp5O_L
Sp5O_R
Sp5C_L
Sp5C_R
PBG_L
PBG_R
10N_L
10N_R
VCA_L
VCA_R
VCP_L
VCP_R
DC_L
DC_R
AP
12N
RtTg
SC
IC
